In [1]:
import os

In [2]:
%pwd

'd:\\MLOps Udemy Krish Naik\\DS-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\MLOps Udemy Krish Naik\\DS-Project'

In [36]:
# this is a entity file
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [37]:
# this is the configuaration management
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_dirs
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_fileppath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_fileppath)
        self.schema=read_yaml(schema_filepath)

        create_dirs([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_dirs([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [38]:
from urllib import request

In [39]:
from urllib import request
import zipfile
import os
from src.datascience import logger
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    # function for downloading zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers= request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with followig info: \n{headers}")
        else:
            logger.info(f"File already exist!")

    # function for extracting the zip file
    def extract_zip_file(self):
        """
        zipfile_path: str
        this function returns None
        
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [43]:
try:
    config = ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-28 20:43:08,382:INFO: common: yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'> loaded succesfully!]
[2025-03-28 20:43:08,386:INFO: common: yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded succesfully!]
[2025-03-28 20:43:08,387:INFO: common: yaml file: <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='UTF-8'> loaded succesfully!]
[2025-03-28 20:43:08,389:INFO: common: created directory at: artifacts]
[2025-03-28 20:43:08,394:INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-28 20:43:09,182:INFO: 4119887249: artifacts/data_ingestion/data.zip download! with followig info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: